In [ ]:
from classiq import (
    H,
    QArray,
    QBit,
    QCallable,
    QFunc,
    allocate,
    create_model,
    repeat,
    show,
    synthesize,
)


@QFunc
def my_apply_to_all(my_operand: QCallable[QBit], qba: QArray[QBit]) -> None:
    repeat(
        count=qba.len(),
        iteration=lambda index: my_operand(qba[index]),
    )


@QFunc
def my_hadamard_transform(qba: QArray[QBit]):
    my_apply_to_all(lambda qb: H(qb), qba)


@QFunc
def main():
    qba = QArray("qba")
    allocate(4, qba)
    my_hadamard_transform(qba)


model = create_model(main)
quantum_program = synthesize(model)
show(quantum_program)

In [ ]:
from classiq import *

@QFunc
def shift_right(x:Input[QArray[QBit]], y:Output[QArray[QBit]]):
    garbage = QArray('garbage')
    split(1,x.len()-1,x,garbage,y)

@QFunc
def main(y:Output[QNum]):
    x = QNum('x')
    prepare_int(9,x) #9=1001
    shift_right(x,y) #9>>1 = 100 = 4


qmod = create_model(main)
qprog= synthesize(qmod)
show(qprog)

In [ ]:
from sympy import pi
from classiq import RX, H, QNum, QCallable, QFunc, QParam, allocate_num, create_model, show, synthesize


@QFunc
def my_operator(my_operand: QCallable[QParam[float], QNum]) -> None:
    num = QNum("num")
    allocate_num(4,False,0,num)
    my_operand(0.5, num)

@QFunc
def main() -> None:
    def my_operand_func(t, v):
        v = v + t

    my_operator(my_operand_func)

qmod = create_model(main)
qprog= synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/2208acef-9112-493c-9ff5-34060960f289?version=0.36.1


In [ ]:
from classiq import (
    QNum,
    QCallable, 
    QFunc, 
    QParam, 
    allocate_num, 
    create_model, 
    show, 
    synthesize, 
    QArray, 
    Output, 
    hadamard_transform, 
    repeat, 
    control,
    inplace_add,
    factorial,
    bernoulli,
    power
)

precision = 4



@QFunc
def calc_element(
    n: QParam[int], x:QNum, num: Output[QNum]
    
) -> None:
    num |= ((2**(2*n))*(2**(2*n)-1)*bernoulli(2*n)*(x**((2*n)-1)))/factorial(2*n)

@QFunc
def inplace_calc_element(
    n: QParam[int], x:QNum, target: Output[QNum]
    
) -> None:
    tmp = QNum('tmp')
    within_apply(compute= lambda: calc_element(n,x,tmp),
                action= lambda: inplace_add(tmp,target))


@QFunc
def calc_elements(
    x : QNum, target: Output[QNum]
) -> None:

    repeat(
        count=precision,
        iteration=lambda index: element_numerator(index,x,target)
        )

@QFunc
def main(x:Output[QNum], target: Output[QNum]) -> None:
    allocate_num(num_qubits=precision, is_signed=False, fraction_digits=precision,out=x)
    hadamard_transform(x)
    calc_elements(x, target)

qmod = create_model(main)

qprog= synthesize(qmod)
show(qprog)

ValidationError: 1 validation error for Model
__root__
  Handle 'target' was errored with "At the end of the function, in port target is expected to be HandleState.INITIALIZED but it isn't"
Handle 'num' was errored with "Trying to access handle 'num' as output but it is in incorrect state"
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack
If the error persists feel free to open a ticket at: https://short.classiq.io/support (type=value_error.classiqvalue; _raw_message=Handle 'target' was errored with "At the end of the function, in port target is expected to be HandleState.INITIALIZED but it isn't"
Handle 'num' was errored with "Trying to access handle 'num' as output but it is in incorrect state")

In [ ]:
from classiq import Model, RegisterUserInput, synthesize, show
from classiq.builtin_functions import Arithmetic

expression = "a*b==c"
definitions = dict(a=RegisterUserInput(size=5), b=RegisterUserInput(size=5),c=RegisterUserInput(size=5))
oracle_params = Arithmetic(expression=expression, definitions=definitions)
model = Model()
model.Arithmetic(oracle_params)
quantum_program = synthesize(model.get_model())
show(quantum_program)

Opening: https://platform.classiq.io/circuit/098aa64e-2d6c-4c77-b1fb-a1a2749af0e7?version=0.36.1


In [ ]:
from classiq import H, Output, QArray, QFunc, QNum, allocate, bind, repeat, invert


@QFunc
def roll_d8(res: Output[QNum]) -> None:
    d8 = QArray("d8")
    allocate(3, d8)
    repeat(3, lambda i: H(d8[i]))
    bind(d8, res)  # res <- d8
    invert(operand=lambda:bind(d8,res))

    
qmod = create_model(roll_d8)

qprog= synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/d47c5ed7-121f-4bd9-9670-3a37286cc5e8?version=0.36.1
